In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings

import io
from urllib.request import urlopen

warnings.filterwarnings("ignore")
url='http://www.preqveca.ru/placements/?sf%5Bipo_t%5D=&sf%5Bipo%5D=0&sf%5Bstatus%5D=0&sf%5Bcountr%5D=0&sf%5Bspec%5D=0&sf%5Blisting%5D=0&sf%5Bpsf%5D=&sf%5Bpst%5D=&sf%5Bpt%5D=0&sf%5Bind%5D=0&sf%5Bpef%5D=&sf%5Bpet%5D=&rec_start=30&rec_start=0'
respone = requests.get(url, verify=False)
result=pd.read_html(url)[2]

for i in range(12):
    url = 'http://www.preqveca.ru/placements/?sf%5Bipo_t%5D=&sf%5Bipo%5D=0&sf%5Bstatus%5D=0&sf%5Bcountr%5D=0&sf%5Bspec%5D=0&sf%5Blisting%5D=0&sf%5Bpsf%5D=&sf%5Bpst%5D=&sf%5Bpt%5D=0&sf%5Bind%5D=0&sf%5Bpef%5D=&sf%5Bpet%5D=&rec_start=30&rec_start=0' + '&rec_start='+str(30*(i+1))
    response = requests.get(url, verify=False)
    tables = pd.read_html(url)[2]
    result = pd.concat([result, tables], ignore_index=True)
result
#Тут мы запарсили основную табличку с данными с 13 страниц сайта

,Название IPO / SPO,Отрасль,Биржа размещения,"Объем привлеченных средств, млн. USD",Дата окончания размещения
0,ВТБ (SPO-4),Финансовые услуги,ММВБ,1 157,07.06.2023
1,Genetico,Биотехнологии и медико-биологические исследова...,ММВБ,2,25.04.2023
2,Whoosh,"Технологии, телекоммуникации и медиа, Транспор...",ММВБ,33,14.12.2022
3,КазМунайГаз,Нефть и газ,"Astana International Exchange (AIX), KASE",330,02.12.2022
4,Positive Technologies (SPO),"Технологии, телекоммуникации и медиа",ММВБ,17,28.09.2022
...,...,...,...,...,...
368,Kaspi.kz (SPO),NaN,London Stock Exchange,366,15.03.2021
369,Европлан (IPO-II),"Транспорт и логистика, Финансовые услуги",ММВБ,NaN,NaN
370,Mercury Retail Group (MRG),"FMCG, Торговля и потребительский сектор",ММВБ,1 300,NaN
371,Алмазы Арктики (АЛМАР),Металлургия и горное дело,Санкт-Петербургская биржа,9,NaN


На сайте 13 страниц с данными о сделках, также каждой сделке отведена отдельная страница с более детальной информацией.
Мы запарсили сами 13 страниц, поняв закономерность, по которой меняется url сайта.

In [5]:
url2 = 'http://www.preqveca.ru/placements/570/'
df=pd.read_html(url2)[1]
df = df.T
df = df.rename(columns=df.iloc[0])
df = df.drop(0, axis = 0)



for i in range(13):
    url = 'http://www.preqveca.ru/placements/?sf%5Bipo_t%5D=&sf%5Bipo%5D=0&sf%5Bstatus%5D=0&sf%5Bcountr%5D=0&sf%5Bspec%5D=0&sf%5Blisting%5D=0&sf%5Bpsf%5D=&sf%5Bpst%5D=&sf%5Bpt%5D=0&sf%5Bind%5D=0&sf%5Bpef%5D=&sf%5Bpet%5D=&rec_start=30&rec_start=0' + '&rec_start='+str(30*(i))

    pages_to_parse=[url]

    while pages_to_parse != []:
        current_page = pages_to_parse.pop()
        html_doc = urlopen(url).read()
        soup = BeautifulSoup(html_doc, 'html.parser')
        # ищем div с нужным классом
        collect = soup.find('table', {'class': 'datagrid'})
        # в найденном div'е, ищем все ссылки (т.е. теги a).
        a_tags_on_page = collect.findAll('a') 
        links_on_page = []
        for link in a_tags_on_page:
            links_on_page.append(str(link.get('href')))
    links_on_page

    for j in range(1, len(links_on_page)):
        url3 = 'http://www.preqveca.ru' + links_on_page[j]
        df1=pd.read_html(url3)[1]
        df1 = df1.T
        df1 = df1.rename(columns=df1.iloc[0])
        df1 = df1.drop(0, axis = 0)
        df = pd.concat([df, df1],ignore_index=True)

df

,Название,Спецификация размещения,Тип размещения,Тип подписки,Начало,Статус,Окончание,Количество размещённых ц.б.,"Объем привлеченных средств, млн. USD","Цена размещения, USD",Биржа размещения,Инвестиционные банки,Страна операций,Отрасль,"Ценовой коридор, верхняя граница, USD","Ценовой коридор, нижняя граница, USD","Капитализация на момент размещения, млн. USD",Проспект эмиссии,Рост/Падение с даты размещения,Offering Memorandum
0,Genetico,акция обыкновенная,IPO,Открытый,12.04.2023,Завершено,25.04.2023,10 000 000,2,0.22,ММВБ,"АЛОР ИНВЕСТ, ИВА Партнерс",Россия,Биотехнологии и медико-биологические исследова...,NaN,NaN,NaN,NaN,NaN,NaN
1,Genetico,акция обыкновенная,IPO,Открытый,12.04.2023,Завершено,25.04.2023,10 000 000,2,0.22,ММВБ,"АЛОР ИНВЕСТ, ИВА Партнерс",Россия,Биотехнологии и медико-биологические исследова...,NaN,NaN,NaN,NaN,NaN,NaN
2,Whoosh,акция обыкновенная,IPO,Открытый,05.12.2022,Завершено,14.12.2022,NaN,33,2.92,ММВБ,"Сбербанк, Тинькофф банк","Беларусь, Казахстан, Россия","Технологии, телекоммуникации и медиа, Транспор...",3.56,2.92,326,скачать8163Кб,NaN,NaN
3,КазМунайГаз,акция обыкновенная,IPO,Открытый,23.11.2022,Завершено,02.12.2022,18 296 455,330,17.78,"Astana International Exchange (AIX), KASE",Freedom Finance,Казахстан,Нефть и газ,NaN,NaN,11 000,NaN,NaN,NaN
4,Positive Technologies (SPO),акция обыкновенная,SPO,Открытый,19.09.2022,Завершено,28.09.2022,932 600,17,18.25,ММВБ,БКС-банк,Россия,"Технологии, телекоммуникации и медиа",22.00,20.00,1 234,NaN,18.64%,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,Kaspi.kz (SPO),GDR,SPO,Открытый,NaN,Завершено,15.03.2021,6 000 000,366,61.00,London Stock Exchange,"Citigroup, Morgan Stanley",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,Европлан (IPO-II),акция обыкновенная,IPO,Открытый,NaN,Планируемый,NaN,NaN,NaN,NaN,ММВБ,NaN,NaN,"Транспорт и логистика, Финансовые услуги",NaN,NaN,NaN,NaN,NaN,NaN
358,Mercury Retail Group (MRG),GDR,IPO,Открытый,NaN,Отложено,NaN,200 000 000,1 300,NaN,ММВБ,"Bank of America, JP Morgan, ВТБ Капитал, Газпр...",Россия,"FMCG, Торговля и потребительский сектор",6.50,6.00,13 000,NaN,NaN,NaN
359,Алмазы Арктики (АЛМАР),акция обыкновенная,IPO,Открытый,NaN,Планируемый,NaN,21 805,9,NaN,Санкт-Петербургская биржа,"ITI Capital, ИФК Солид",Россия,Металлургия и горное дело,NaN,NaN,NaN,скачать9592Кб,NaN,NaN


Тут мы достали отдельно данные по каждой сделке, переходя по каждой из них и выбирая таблицу нужного класса, который мы нашли в коде страницы. Далее мы вытащили номер сделки на сайте (href) и циклом подгрузили в табличку

In [7]:
print(set(result['Название IPO / SPO'].unique())-set(df['Название'].unique()))
print()
print(set(df['Название'].unique())-set(result['Название IPO / SPO'].unique()))

{'ММК', 'Росинтер Ресторантс / SPO-1', 'Иркут', 'ВТБ (SPO-4)', 'КДД Группа', 'Совкомфлот', 'КОКС', 'Верофарм', 'ВТБ (SPO)-3', 'Лэндком', 'Европлан (SPO, делистинг)', 'ФосАгро', 'Мироновский хлебопродукт / SPO'}

set()


Оказалось, что не все сделки попали в таблицу, из-за разницы в коде сайта. Руками нашли их и занесли.

In [10]:
allin=df

helpme=[76, 149, 407, 80, 571, 98, 371, 213, 129, 221, 265, 33, 219]
for i in helpme:
    url2 = 'http://www.preqveca.ru/placements/' + str(i)
    tables =pd.read_html(url2, index_col=0)[1].T
    allin = pd.concat([allin, tables], ignore_index=True)
allin

,Название,Спецификация размещения,Тип размещения,Тип подписки,Начало,Статус,Окончание,Количество размещённых ц.б.,"Объем привлеченных средств, млн. USD","Цена размещения, USD",Биржа размещения,Инвестиционные банки,Страна операций,Отрасль,"Ценовой коридор, верхняя граница, USD","Ценовой коридор, нижняя граница, USD","Капитализация на момент размещения, млн. USD",Проспект эмиссии,Рост/Падение с даты размещения,Offering Memorandum
0,Genetico,акция обыкновенная,IPO,Открытый,12.04.2023,Завершено,25.04.2023,10 000 000,2,0.22,ММВБ,"АЛОР ИНВЕСТ, ИВА Партнерс",Россия,Биотехнологии и медико-биологические исследова...,NaN,NaN,NaN,NaN,NaN,NaN
1,Genetico,акция обыкновенная,IPO,Открытый,12.04.2023,Завершено,25.04.2023,10 000 000,2,0.22,ММВБ,"АЛОР ИНВЕСТ, ИВА Партнерс",Россия,Биотехнологии и медико-биологические исследова...,NaN,NaN,NaN,NaN,NaN,NaN
2,Whoosh,акция обыкновенная,IPO,Открытый,05.12.2022,Завершено,14.12.2022,NaN,33,2.92,ММВБ,"Сбербанк, Тинькофф банк","Беларусь, Казахстан, Россия","Технологии, телекоммуникации и медиа, Транспор...",3.56,2.92,326,скачать8163Кб,NaN,NaN
3,КазМунайГаз,акция обыкновенная,IPO,Открытый,23.11.2022,Завершено,02.12.2022,18 296 455,330,17.78,"Astana International Exchange (AIX), KASE",Freedom Finance,Казахстан,Нефть и газ,NaN,NaN,11 000,NaN,NaN,NaN
4,Positive Technologies (SPO),акция обыкновенная,SPO,Открытый,19.09.2022,Завершено,28.09.2022,932 600,17,18.25,ММВБ,БКС-банк,Россия,"Технологии, телекоммуникации и медиа",22.00,20.00,1 234,NaN,18.64%,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,Иркут,акция обыкновенная,IPO,Открытый,15.03.2004,Завершено,26.03.2004,205 000 000,127,0.62,"ММВБ, РТС",МДМ-Банк,Россия,Машиностроение,0.68,0.54,522,NaN,-49.19%,скачать3494Кб
370,КОКС,"GDR, акция обыкновенная",IPO,Открытый,24.01.2011,Отложено,03.02.2011,NaN,NaN,NaN,"London Stock Exchange, ММВБ, РТС","Citigroup, UBS Investment Bank, ВТБ Капитал",Россия,Металлургия и горное дело,8.00,6.25,NaN,NaN,NaN,NaN
371,ФосАгро,GDR,IPO,Открытый,26.06.2011,Завершено,13.07.2011,1 280 952,538,14.00,London Stock Exchange,"Citigroup, Ренессанс Капитал",Россия,Химия и нефтехимия,16.50,13.00,5 200,NaN,NaN,NaN
372,ММК,"GDR, акция обыкновенная",IPO,Открытый,09.04.2007,Завершено,30.04.2007,1 040 000 000,1 121,0.96,"London Stock Exchange, ММВБ","ABN AMRO, Morgan Stanley, Газпромбанк, Ренесса...",Россия,Металлургия и горное дело,1.19,0.94,11 221,NaN,-75.14%,скачать1776Кб


In [9]:
print(set(result['Название IPO / SPO'].unique())-set(allin['Название'].unique()))
print()
print(set(allin['Название'].unique())-set(result['Название IPO / SPO'].unique()))

set()

set()


In [12]:
result.to_csv('result', sep='\t',index=False)
allin.to_csv('result 2', sep='\t',index=False)